## Python descriptors

In [1]:
class MyClass:
    attr = 1
    
    def __init__(self, attr):
        self.attr = attr

In [2]:
inst = MyClass("test")

In [3]:
inst.__dict__

{'attr': 'test'}

In [4]:
type(inst.__dict__)

dict

In [5]:
inst.__dict__['attr'] == inst.attr 

True

In [6]:
inst.__class__

__main__.MyClass

In [7]:
inst.__class__.__dict__

mappingproxy({'__module__': '__main__',
              'attr': 1,
              '__init__': <function __main__.MyClass.__init__(self, attr)>,
              '__dict__': <attribute '__dict__' of 'MyClass' objects>,
              '__weakref__': <attribute '__weakref__' of 'MyClass' objects>,
              '__doc__': None})

In [8]:
inst.__class__.__dict__['attr']

1

In [9]:
inst.__class__.__dict__['attr'] == inst.__class__.attr

True

In [10]:
inst.__class__.__dict__['attr'] == MyClass.attr

True

In [11]:
class MyBehaviorClass(MyClass):
        
    def print_attr(self):
        return self.attr

In [12]:
MyBehaviorClass.__dict__

mappingproxy({'__module__': '__main__',
              'print_attr': <function __main__.MyBehaviorClass.print_attr(self)>,
              '__doc__': None})

In [13]:
inst = MyBehaviorClass("test")

In [14]:
inst.__dict__

{'attr': 'test'}

In [15]:
inst.__class__.__dict__

mappingproxy({'__module__': '__main__',
              'print_attr': <function __main__.MyBehaviorClass.print_attr(self)>,
              '__doc__': None})

In [16]:
inst.print_attr()

'test'

In [17]:
MyBehaviorClass.print_attr()

TypeError: print_attr() missing 1 required positional argument: 'self'

In [18]:
MyBehaviorClass.print_attr(inst)

'test'

In [19]:
inst.attr

'test'

In [24]:
del MyClass.attr

In [25]:
inst.__dict__

{}

In [26]:
inst.attr

AttributeError: 'MyBehaviorClass' object has no attribute 'attr'

So, when we try to access a particular name inside the instance of class, three rules are stacked in lookup chain by Python interpreter:
   - look for a name inside instance dict `self.__dict__[name]`
   - look for a name inside base class for instance `self.__class__.__dict__[name]`
   - look for a name inside base classes of type(a) `self.__class__.__mro__`

However, if the looked-up value is an object defining one of the descriptor methods, then Python may override the default behavior and invoke the descriptor method instead. Where this occurs in the precedence chain depends on which descriptor methods were defined and how they were called.

In general, a descriptor is an object attribute with “binding behavior”, one whose attribute access has been overridden by methods in the descriptor protocol: `__get__()`, `__set__()`, and `__delete__()`. If any of those methods are defined for an object, it is said to be a descriptor.

When we have descriptors, the lookup chain is modified with additional rules like this:
 - if `self.__class__.__dict__.get(name)` is <b>data descriptor</b> and has `__get__` method return `self.__class__.__dict__[name].__get__(instance, self.__class__)` else look for a name inside instance dict `self.__dict__[name]`
 - if `self.__class__.__dict__.get(name)` is <b>non data descriptor</b> return `self.__class__.__dict__[name].__get__(instance, self.__class__)` else look for a name inside base class for instance `self.__class__.__dict__[name]`
 - repeat previous step for base classes for type(a) `self.__class__.__mro__`

<slot wrapper '__getattribute__' of 'object' objects>

In [27]:
class MyAttr:
    
    def __get__(self, instance, owner):
        print('get invoked')
        return 42

    def __set__(self, instance, value):
        print('set invoked')
    

In [28]:
class Base:
    ...

In [29]:
Base.a = 1

In [30]:
Base.d = MyAttr()

In [31]:
Base.d

get invoked


42

In [32]:
Base.__dict__

mappingproxy({'__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'Base' objects>,
              '__weakref__': <attribute '__weakref__' of 'Base' objects>,
              '__doc__': None,
              'a': 1,
              'd': <__main__.MyAttr at 0x107790210>})

In [33]:
b = Base()

In [34]:
b.d = 20

set invoked


In [35]:
b.d

get invoked


42

In [36]:
b.__dict__

{}

In [37]:
b.__class__.__mro__

(__main__.Base, object)

In [38]:
Base.d = 1

In [39]:
Base.__dict__

mappingproxy({'__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'Base' objects>,
              '__weakref__': <attribute '__weakref__' of 'Base' objects>,
              '__doc__': None,
              'a': 1,
              'd': 1})

In [40]:
b2 = Base()

In [41]:
b.d = 20

In [42]:
b.d

20

In [43]:
class MyAttr:

    def __init__(self, name):
        self.name = name

    def __get__(self, instance, owner):
        print('get invoked')
        if instance is None:
            return self
        return instance.__dict__[self.name]
    
    def __set__(self, instance, value):
        print('set invoked')
        if value > 0:
            instance.__dict__[self.name] = value
            return
        raise ValueError('value must be > 0')

In [44]:
class ValidContainer:
    x = MyAttr('x')
    y = MyAttr('y')
    
    def __init__(self, a, b):
        self.x = a
        self.y = b
    

In [45]:
q = ValidContainer(1, 2)

set invoked
set invoked


In [46]:
q.x

get invoked


1

In [47]:
q.__dict__

{'x': 1, 'y': 2}

In [ ]:
q.x = 2
q.x

In [48]:
class MyAttr:

    def __init__(self, name):
        self.name = name
    
    def __set__(self, instance, value):
        print('set invoked')
        if value > 0:
            instance.__dict__[self.name] = value
            return
        raise ValueError('value must be > 0')
        
class ValidContainer:
    x = MyAttr('x')
    y = MyAttr('y')
    
    def __init__(self, a, b):
        self.x = a
        self.y = b


In [49]:
q = ValidContainer(1, 2)

set invoked
set invoked


In [50]:
q.x

1

In [51]:
import time
    
    
class LongJourney:
    
    def __init__(self):
        self.val = self.calc_val()
    
    def calc_val(self):
        time.sleep(1)
        return 42

In [52]:
x = LongJourney()
y = LongJourney()
z = LongJourney()

In [53]:
x.val

42

In [54]:
class MyAttr:

    def __init__(self, name):
        self.name = name

    def __get__(self, instance, owner):
        print('invoked')
        if instance is None:
            return self
        val = instance.calc_val()
        instance.__dict__[self.name] = val
        return val
    
class LongJourney:
    val = MyAttr('val')
    
    def calc_val(self):
        time.sleep(1)
        return 42

In [55]:
x = LongJourney()
y = LongJourney()
z = LongJourney()

In [56]:
print(x.val)

invoked
42


In [62]:
print(x.val)

42


In [64]:
class ShowDescr:
    
    def test(self, a):
        return self + a

In [65]:
ShowDescr.__dict__

mappingproxy({'__module__': '__main__',
              'test': <function __main__.ShowDescr.test(self, a)>,
              '__dict__': <attribute '__dict__' of 'ShowDescr' objects>,
              '__weakref__': <attribute '__weakref__' of 'ShowDescr' objects>,
              '__doc__': None})

In [66]:
ShowDescr.test

<function __main__.ShowDescr.test(self, a)>

In [67]:
ShowDescr().test

<bound method ShowDescr.test of <__main__.ShowDescr object at 0x1092a5b90>>

In [68]:
x = ShowDescr.test.__get__(2)
x

<bound method ShowDescr.test of 2>

In [69]:
x(2)

4